In [ ]:
!wget -O RNN-HAR-2D-Pose-database.zip https://drive.google.com/file/d/1ghTRzZW4nx70K-iARGWi7jaWzHxekvCS/view?usp=sharing

In [ ]:
!unzip RNN-HAR-2D-Pose-database.zip

In [ ]:
!head -2 /content/RNN-HAR-2D-Pose-database/X_train.txt

In [ ]:
!git clone https://github.com/Paromita23/Human_Action_Recognition_using_deep_learning_algorithm.git

In [ ]:
%cd /Human-Action-Recognition-Using-Detectron2-And-Lstm

In [ ]:
!pip install -r requirements.txt

In [ ]:
DATASET_PATH = "/content/RNN-HAR-2D-Pose-database/"

In [ ]:
from argparse import ArgumentParser

def configuration_parser(parent_parser):
    parser = ArgumentParser(parents=[parent_parser], add_help=False)
    parser.add_argument('--batch_size', type=int, default=512)
    parser.add_argument('--epochs', type=int, default=400)
    parser.add_argument('--data_root', type=str, default=DATASET_PATH)
    parser.add_argument('--learning_rate', type=float, default=0.0001)
    parser.add_argument('--num_class', type=int, default=6)
    return parser

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor

from src.lstm import ActionClassificationLSTM, PoseDataModule

In [ ]:
def do_training_validation():
    pl.seed_everything(21)    
    parser = ArgumentParser()
    parser = pl.Trainer.add_argparse_args(parser)
    parser = configuration_parser(parser)
    # args = parser.parse_args()
    args, unknown = parser.parse_known_args()
    # init model    
    model = ActionClassificationLSTM(34, 50, learning_rate=args.learning_rate)
    data_module = PoseDataModule(data_root=args.data_root,
                                        batch_size=args.batch_size)    
    #save only the top 1 model based on val_loss
    checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor='val_loss')
    lr_monitor = LearningRateMonitor(logging_interval='step')  
    #trainer
    trainer = pl.Trainer.from_argparse_args(args,
        # fast_dev_run=True,
        max_epochs=args.epochs, 
        deterministic=True, 
        gpus=1, 
        progress_bar_refresh_rate=1, 
        callbacks=[EarlyStopping(monitor='train_loss', patience=15), checkpoint_callback, lr_monitor])    
    trainer.fit(model, data_module)    
    return model

In [ ]:
# To reload tensorBoard
%load_ext tensorboard

# logs folder path
%tensorboard --logdir=lightning_logs